In [1]:
!pwd
!ls

/home/chen/dev/Factory/Qwen2.5-VL
cookbooks	  image2textVL.ipynb  README.md
docker		  LICENSE	      requirements_web_demo.txt
downloadmodel.py  qwen-vl-finetune    web_demo_mm.py
evaluation	  qwen-vl-utils       web_demo_streaming


In [2]:
# from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
# from qwen_vl_utils import process_vision_info
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # default: Load the model on the available device(s)
# # model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
# #     "Qwen/Qwen2.5-VL-3B-Instruct-AWQ", 
# #     torch_dtype= torch.float16, 
# #     device_map="auto",
# #     # attn_implementation="sdpa",
# # )
# # print dtype and device map
# # print(f"Model dtype: {model.dtype}")
# # print(f"Model device map: {model.hf_device_map}")

# # We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-3B-Instruct-AWQ",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# # default processer
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct-AWQ")

# # The default range for the number of visual tokens per image in the model is 4-16384.
# # You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# # min_pixels = 256*28*28
# # max_pixels = 1280*28*28
# # processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct-AWQ", min_pixels=min_pixels, max_pixels=max_pixels)

# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
#             },
#             {"type": "text", "text": "Describe this image."},
#         ],
#     }
# ]

# # Preparation for inference
# text = processor.apply_chat_template(
#     messages, tokenize=False, add_generation_prompt=True
# )
# image_inputs, video_inputs = process_vision_info(messages)
# inputs = processor(
#     text=[text],
#     images=image_inputs,
#     videos=video_inputs,
#     padding=True,
#     return_tensors="pt",
# )
# inputs = inputs.to("cuda")

# # Inference: Generation of the output
# generated_ids = model.generate(**inputs, max_new_tokens=128)
# generated_ids_trimmed = [
#     out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
# ]
# output_text = processor.batch_decode(
#     generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
# )
# print(output_text)


In [3]:
from awq.models.base import BaseAWQForCausalLM
# from transformers import AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor
# 使用 AWQ 量化模型路径
model_name = "Qwen/Qwen2.5-VL-3B-Instruct-AWQ"

# 加载 tokenizer 和 processor
tokenizer = AutoTokenizer.from_pretrained(model_name)
processor = AutoProcessor.from_pretrained(model_name)

# # 加载量化模型
# model = AutoModelForCausalLM.from_quantized(
#     model_name,
#     device_map="auto",
#     fuse_layers=True,
#     trust_remote_code=False,
#     safetensors=True
# )

from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(model_name, attn_implementation="flash_attention_2", device_map="cuda:0")

# 打印模型信息
print(f"Model torch_dtype: {model.dtype}")

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg", 
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# 准备推理输入
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# 推理生成输出
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

/home/chen/dev/.venv/lib/python3.13/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will re

ValueError: Unrecognized configuration class <class 'transformers.models.qwen2_5_vl.configuration_qwen2_5_vl.Qwen2_5_VLConfig'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of AriaTextConfig, BambaConfig, BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BitNetConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CohereConfig, Cohere2Config, CpmAntConfig, CTRLConfig, Data2VecTextConfig, DbrxConfig, DeepseekV3Config, DiffLlamaConfig, ElectraConfig, Emu3Config, ErnieConfig, FalconConfig, FalconMambaConfig, FuyuConfig, GemmaConfig, Gemma2Config, Gemma3Config, Gemma3TextConfig, GitConfig, GlmConfig, Glm4Config, GotOcr2Config, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, GraniteConfig, GraniteMoeConfig, GraniteMoeHybridConfig, GraniteMoeSharedConfig, HeliumConfig, JambaConfig, JetMoeConfig, LlamaConfig,

In [ ]:

# from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
# from qwen_vl_utils import process_vision_info

# # default: Load the model on the available device(s)
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     # "Qwen/Qwen2.5-VL-7B-Instruct",
#     "Qwen/Qwen2.5-VL-3B-Instruct", 
#     torch_dtype="auto", 
#     device_map="auto"
# )

# # print model torch_dtype and device_map
# print(f"Model torch_dtype: {model.dtype}")
# # print(f"Model device_map: {model.device_map}")

# # We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# # model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
# #     "Qwen/Qwen2.5-VL-7B-Instruct",
# #     torch_dtype=torch.bfloat16,
# #     attn_implementation="flash_attention_2",
# #     device_map="auto",
# # )

# # default processor
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# # The default range for the number of visual tokens per image in the model is 4-16384.
# # You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# # min_pixels = 256*28*28
# # max_pixels = 1280*28*28
# # processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
#             },
#             {"type": "text", "text": "Describe this image."},
#         ],
#     }
# ]

# # Preparation for inference
# text = processor.apply_chat_template(
#     messages, tokenize=False, add_generation_prompt=True
# )
# image_inputs, video_inputs = process_vision_info(messages)
# inputs = processor(
#     text=[text],
#     images=image_inputs,
#     videos=video_inputs,
#     padding=True,
#     return_tensors="pt",
# )
# inputs = inputs.to(model.device)

# # Inference: Generation of the output
# generated_ids = model.generate(**inputs, max_new_tokens=128)
# generated_ids_trimmed = [
#     out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
# ]
# output_text = processor.batch_decode(
#     generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
# )
# print(output_text)

In [ ]:
# import torch
# from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
# from qwen_vl_utils import process_vision_info

# # 加载模型：启用 4-bit 量化（NF4 类型）
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct",
#     device_map="auto",                      # 自动分配设备（支持多 GPU）
#     load_in_4bit=True,                      # 启用 4-bit 量化
#     bnb_4bit_quant_type="nf4",              # NF4 是一种适合预训练权重的量化类型
#     bnb_4bit_use_double_quant=True,         # 双重量化进一步压缩
#     bnb_4bit_compute_dtype=torch.bfloat16   # 计算时使用 bfloat16 提升性能
# )

# # 加载处理器（processor）
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# # 示例输入消息
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg", 
#             },
#             {"type": "text", "text": "Describe this image."},
#         ],
#     }
# ]

# # 预处理文本部分
# text = processor.apply_chat_template(
#     messages, tokenize=False, add_generation_prompt=True
# )

# # 处理视觉信息（提取图像/视频）
# image_inputs, video_inputs = process_vision_info(messages)

# # 构建最终输入张量
# inputs = processor(
#     text=[text],
#     images=image_inputs,
#     videos=video_inputs,
#     padding=True,
#     return_tensors="pt",
# )
# inputs = inputs.to(model.device)  # 移动到模型所在设备（如 GPU）

# # 生成输出
# generated_ids = model.generate(**inputs, max_new_tokens=128)

# # 去除 prompt 部分，仅保留生成内容
# generated_ids_trimmed = [
#     out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
# ]

# # 解码输出
# output_text = processor.batch_decode(
#     generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
# )

# # 打印结果
# print(output_text)

In [ ]:
# 示例输入消息
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg", 
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# 预处理文本部分
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

# 处理视觉信息（提取图像/视频）
image_inputs, video_inputs = process_vision_info(messages)

# 构建最终输入张量
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)  # 移动到模型所在设备（如 GPU）

# 生成输出
generated_ids = model.generate(**inputs, max_new_tokens=128)

# 去除 prompt 部分，仅保留生成内容
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

# 解码输出
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

# 打印结果
print(output_text)

In [ ]:
user_prompt = """Describe object(s) and (possible) their relative locations in brief with 1 very short sentence:
                DO NOT describe background/surface: 
                for example: 
                A red phone is under a wood mouse. ; 
                Only one red cup. ; 
                The image contains no visible objects""" 

# 示例输入消息
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "/home/chen/dev/Factory/img/GreenUp1.jpg", 
            },
            {"type": "text", "text": user_prompt},
        ],
    }
]

# 预处理文本部分
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

# 处理视觉信息（提取图像/视频）
image_inputs, video_inputs = process_vision_info(messages)

# 构建最终输入张量
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)  # 移动到模型所在设备（如 GPU）

# 生成输出
generated_ids = model.generate(**inputs, max_new_tokens=128)

# 去除 prompt 部分，仅保留生成内容
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

# 解码输出
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

# 打印结果
print(output_text)